In [19]:
import pandas as pd
import json

with open('demo-json.json', 'r') as file:
    data = json.load(file)  # Load JSON file into a dictionary or list

In [5]:
pip install --upgrade snowflake-connector-python pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
import snowflake.connector
import os
import requests
PASSWORD = os.getenv('SNOWSQL_PWD')
print(PASSWORD)

5eWxWv4EvyCkhkY


In [125]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password=PASSWORD,
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS tiny_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS testschema_mg")
        cs.execute("USE WAREHOUSE tiny_warehouse_mg")
        cs.execute("USE SCHEMA testschema_mg")
        runQueries(cs)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

CREATE OR REPLACE TABLE article_table_pandas_3 (product_sn STRING, name STRING, category STRING, price_amount STRING, price_currency STRING, price_discount_percentage STRING, price_discount_valid_until STRING, availability_status_in_stock STRING, availability_status_quantity STRING)
INSERT INTO article_table_pandas_3 (product_sn, name, category, price_amount, price_currency, price_discount_percentage, price_discount_valid_until, availability_status_in_stock, availability_status_quantity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
INSERT INTO article_table_pandas_3 (product_sn, name, category, price_amount, price_currency, price_discount_percentage, price_discount_valid_until, availability_status_in_stock, availability_status_quantity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
INSERT INTO article_table_pandas_3 (product_sn, name, category, price_amount, price_currency, price_discount_percentage, price_discount_valid_until, availability_status_in_stock, availability_status_quantity) V

In [123]:
def runQueries(cs):    
    # Extract product list
    products = data["products"]
    flattened_data = [flatten_json(product) for product in products]
    
    # Convert to Pandas DataFrame
    df = pd.DataFrame(flattened_data)
    # Rename specific columns
    df = df.rename(columns={"id": "product_sn"})
    df["product_sn"] = df["product_sn"].astype(str)
    df = df.astype(str).apply(lambda col: col.map(lambda x: None if pd.isna(x) or x == "" or str(x).strip().lower() == "nan" else x))

    # Generate CREATE TABLE statement dynamically
    table_name = "article_table_pandas_3"
    columns = ", ".join([f'{col} STRING' for col in df.columns])
    create_table_sql = f'CREATE OR REPLACE TABLE {table_name} ({columns})'
    print(create_table_sql)
    cs.execute(create_table_sql)

    # Insert data into Snowflake
    for _, row in df.iterrows():
        placeholders = ", ".join(["%s"] * len(row))
        insert_sql = f'INSERT INTO {table_name} ({", ".join(df.columns)}) VALUES ({placeholders})'
        cs.execute(insert_sql, tuple(row))

In [57]:
# Flatten JSON recursively
def flatten_json(nested_json, parent_key=""):
    items = {}
    for k, v in nested_json.items():
        new_key = f"{parent_key}_{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_json(v, new_key))  # Recurse
        else:
            items[new_key] = v  # Store the value
    return items

In [27]:
def getColumnValue(columns_list,currentRow,columns_string):
    values = [columns_string] + [currentRow[col] for col in columns_list]
    return tuple(values)